# Method 1: Spatial microsimulation

Spatial microsimulation is a method that allows us to create synthetic data to represent a whole population when we don't actually have the data for a whole population. In this case, we're taking data from the British Social Attitudes Survey as our individual granular data, and combining it with census data to create our synthetic data.

### Step 1: Importing all the necessary packages for our code

In [ ]:
import pandas as pd
import numpy as np
import copy
from ipfn import ipfn

### Step 2: Importing our individual-level BSA data and our constraints

In [ ]:
BSA_2020 = pd.read_csv('data/BSA2020/final_bsa_cleaned.csv', low_memory=False)
con_age = pd.read_csv('data/census data/con_age.csv')
con_sex = pd.read_csv('data/census data/con_sex.csv') # 1 represents female, 2 represents male

We also need to check our constraints to ensure that they both have the same population.

In [ ]:
print('con_age total: ',con_age['age_1864'].sum() + con_age['age_65'].sum())
print('con_sex total: ',con_sex['sex_male'].sum() + con_sex['sex_female'].sum())

The above calculations show that both constraints represent the same population size for adults in York.

### Step 3: Preparing the data for spatial microsimulation
First, we will remove all of the BSA_2020 data that isn't related to out constraints. We only need age and sex to be able to allocate individuals to zones so we'll copy the data set and remove all other columns.

In [ ]:
BSA_original = BSA_2020.copy() # making a deep copy

In [ ]:
# dropping all columns unnecessary for weighting
BSA_2020 = BSA_2020.drop(['economic', 
                          'partySup', 
                          'partySupWho',
                          'polInterest',
                          'welfare',
                          'redistrb',
                          'leftright',
                          'leftright2',
                          'libauth',
                          'libauth2',
                          'religion',
                          'nationality',
                          'raceOrigin',
                          'disability',
                          'voteAct',
                          'voteParty'
                         ],axis=1)

Next we need to recategorise the BSA_2020 individual data into bins that match the categories in our constraints dataframe.

In [ ]:
BSA_2020['age'] = pd.to_numeric(BSA_2020['age']) # ensuring age is numeric

# changing age to categorical attribute, with categories matching the indexing of the age constraint
BSA_2020['age'] = pd.cut(BSA_2020['age'], [17,64,120], labels = ['age_1864','age_65'])
BSA_2020.head()

Let's do the same with the sex constraint, and rename the categories in the constraints dataframe to match

In [ ]:
BSA_2020['sex'] = pd.to_numeric(BSA_2020['sex'])

# changing age to categorical attribute, with categories matching the indexing of the age constraint
BSA_2020['sex'] = pd.cut(BSA_2020['sex'], [0,1,2], labels = ['f','m'])
BSA_2020.head()

In [ ]:
BSA_2020['age'].unique()

In [ ]:
con_sex = con_sex.rename(columns={'sex_female':'f','sex_male':'m'}) # all working and looking good!

Next we'll merge these constraints into one table together, merging it on geo_code to avoid duplicate columns and to match up the zones.

In [ ]:
constraints = con_sex.merge(con_age, left_on='geo_code', right_on='geo_code', how='right')

Now that's all merged, but for the next step we also need to drop the geo_code column from the dataframe. We'll make a copy of the constraints dataframe as it is now and then drop the geo_code column from the dataframe we're using.

In [ ]:
constraints_geo = constraints.copy() # making a deep copy

In [ ]:
constraints = constraints.drop(columns=['geo_code'])

In [ ]:
constraints.head()

This gives us one full table with all of our constaint variables, with the names of columns matching the individual level data we have in our BSA_2020 table. The next step to prep the data is to 'flatten' the individual level dataset, so it can be more easily compared with the constraints. This basically means that we turn the data into fields, and have the data entries themselves become boolean. We do this first individually for age and then for sex, and then we combine these into one dataframe.

In [ ]:
# Checking the dimensions of our two dataframes
print ("Shape of our BSA dataframe:",BSA_2020.shape)
print ("Shape of our constraints:",constraints.shape)

So we need these to have the same number of columns. At this time, our BSA dataframe has three columns and our constraints has four.

In [ ]:
# flattening the age column
age_flat = pd.pivot_table(BSA_2020,columns=['age'],values='id', index=BSA_2020.index, aggfunc=len, fill_value=0, observed=False)

# flattening the sex column
sex_flat = pd.pivot_table(BSA_2020,columns=['sex'],values='id', index=BSA_2020.index, aggfunc=len, fill_value=0, observed=False)

In [ ]:
# merge pivoted data to make flatten dataframe
bsa_categorical = pd.DataFrame(age_flat.to_records()).merge(pd.DataFrame(sex_flat.to_records()),left_index=True,right_index=True)

# the above line creates a dataframe with extra index columns, we don
bsa_categorical = bsa_categorical.drop(['index_x','index_y'],axis=1)
bsa_categorical

In [ ]:
# checking the column sums to ensure that this is correct and saving these values for later
bsa_agg = bsa_categorical.sum(axis=0)
bsa_agg # age adds up to 275 (total number of individual data entries) and so does sex so it's worked!

In [ ]:
print ("Shape of individual flattened dataframe:",bsa_categorical.shape)
print ("Shape of constraints dataframe:",constraints.shape)

As we can see when the above code is run, our two dataframe now have the same dimensions and can be compared, as is demonstrated by the code below. The test dataframe created below shows us the first row of the constaints dataframe and the first row of the aggregated dataframe and confirms that both dataframes share the same dimesions and column names.

In [ ]:
test = pd.concat([constraints.iloc[0], bsa_agg], axis=1).transpose()
test

Now our data is all prepped, we can start working towards applying the microsimulation methods to create our synthetic population. 

### Step 4: Iterative Proportional Fitting

Whilst a lot of the coding for IPF can be done with python packages that we can import and use, the general gist of it is to generate weights to show how representative each individual is of each zone.

In [ ]:
BSA_2020_copy = BSA_2020.copy() # making another deep copy of BSA_2020 with the changes we made to variables
BSA_2020_copy

In [ ]:
BSA_2020_copy.info() # looking at data types - we have two category types

In [ ]:
# changing the categorical data into strings - the package we're using doesn't like category data!
BSA_2020_copy['age'] = BSA_2020_copy['age'].astype(str)
BSA_2020_copy['sex'] = BSA_2020_copy['sex'].astype(str)
BSA_2020_copy.info()

Now our BSA_2020_copy data is in the correct datatype, we can use a for loop to add columns with the caluculated weights. Increasing the efficiency from the example provided within the IPF guide page, we can do this with just four lines of code.
1. The first line initiates the loop, and instructs it to repeat for the length of the contstraints dataframe (the number of electoral zones in York).
2. The second line (and the first within the for loop) creates a new column (data-type float) containing 1.0 for each individual to represent the weighting for the current zone.
3. The third carries out the IPF analysis, the key inputs being the dataframe itself, the aggregates (which don't need to be predefined as they can be defined within this line), and the dimensions (the column names we are working with).
4. The final line adds the results of the previous calculation into our BSA_2020_copy dataframe.

In [ ]:
for i in range(0,len(constraints)):
    # creates a column with float value 1.0 to hold that zone's weight
    BSA_2020_copy['weight_' + str(i)] = np.ones(275) 
    
    # carries out the ipf analysis using the dataframe, the aggregates (the constraints database), and the dimensions (sex and age)
    ipf = ipfn.ipfn(BSA_2020_copy, [constraints.iloc[i,[0,1]], constraints.iloc[i,[2,3]]],[['sex'],['age']],weight_col='weight_'+str(i),convergence_rate = 1e-15)
    
    BSA_2020_copy = ipf.iteration()

In [ ]:
BSA_2020_copy # look at all those weights we've calculated!

Now we need to test that these weights we've calculated are actually correct and make sense with what our constraints tell us.

In [ ]:
# creating marginal distribution of individuals 
bsa_agg0 = constraints.apply(lambda x: 1.0*bsa_agg, axis=1)

bsa_agg_final = (bsa_agg0 * np.nan).copy()

for i in range(0,len(constraints)):
    bsa_agg_final.iloc[i] = bsa_categorical.apply(lambda x: x*BSA_2020_copy['weight_'+str(i)],axis=0).sum(axis=0)

bsa_agg_final # this comes out in a funny order so we'll swap the columns to match our constraints for easy comparison

In [ ]:
bsa_agg_final_ordered = bsa_agg_final.reindex(columns=['m', 'f', 'age_1864', 'age_65'])
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reindex.html
bsa_agg_final_ordered

In [ ]:
constraints

The output from our marginal distribution matches our constraints, so we can conclude that the weights we've calculated are correct.

### Step 5: Intergerisation using the TRS method

Intergerisation is the process by which our float values become integers corresponding to the number of individuals within our synthetic population. To do this, we will use the TRS ('trucate replicate sample') method proposed by Lovelace and Ballas (2013).
might want to **rename certain variables here as has been pretty much copied across**

In [ ]:
# intergerisation (Lovelace and Ballas, 2013)
def int_trs(weights):
    xv = np.array(weights).ravel() # convert to vector if necessary
    xint = np.floor(xv) # truncate - get the integer part of the result
    r = xv - xint # get the decimal of the weight
    frac_sum = round(r.sum()) # work out deficit population
    xs = np.random.choice(len(xv),int(frac_sum),True,r/r.sum()) # sample based on deficit
    topup = np.bincount(xs,minlength=len(xv)) # result of the sample deficit
    return xint + topup

In [ ]:
# expansion
def int_expansion(weights):
    return np.repeat(range(0,len(weights)),weights.astype(int))

In [ ]:
# putting this all together!

individuals = [] # creating a list to hold the results of integerisation and expansion

for i in range(0, len(constraints)):
    # calls both functions in one line to integerise and expand
    integerise = int_expansion(int_trs(BSA_2020_copy['weight_' + str(i)]))
    current_ind = BSA_original.loc[integerise].copy() # Select the relevant individuals using .loc
    current_ind['zone'] = i     # Assigning the 'zone' - allocates individuals to different zones
    individuals.append(current_ind)

BSA_2020_synthetic = pd.concat(individuals) # concatonating the resulting list into a new dataframe
BSA_2020_synthetic.reset_index(drop=True, inplace=True) #Resetting the index

In [ ]:
BSA_2020_synthetic # looking at our new synthetic population of York!

Now we'll add the geo_codes column back in to our new synthetic dataframe, so we have a spatial element that we can connect to our shapefile in order to perform our geodemographics!

In [ ]:
constraints_geo_copy = constraints_geo.copy()
constraints_geo = constraints_geo.drop(columns=['m','f','age_1864','age_65'])
constraints_geo

In [ ]:
BSA_synthetics_geo = pd.merge(BSA_2020_synthetic,constraints_geo, left_on='zone', right_index=True, validate='many_to_one')
    # (pandas, 2024)
    # https://pandas.pydata.org/docs/dev/reference/api/pandas.merge.html#pandas.merge

In [ ]:
# let's drop the zone column too for good measyre
BSA_synthetics_geo = BSA_synthetics_geo.drop(columns=['zone'])
BSA_synthetics_geo

Now we have complete synthetic population representing BSA results for the whole of York, and including geo_code zones that we can use to map the data. We'll save our new dataframe as a CVS at this point, and that is the spatial microsimulation part of the project all done!

In [ ]:
BSA_synthetics_geo.to_csv("data/BSA_synthetics_geo.csv", index=False)